# Retail Demo Store Messaging Workshop - Braze

In this workshop we will use [Braze](https://braze.com/) to add the ability to personalize marketing messages to customers of the Retail Demo Store using customer behavioral data and the Personalize models you trained in the prior workshops. We are going to define a campaign in Braze to send target users an email with product recommendations from the Amazon Personalize campaign we created in the Personalization workshop earlier.  You will also create an email template using the Braze Connected Content feature.  When the campaign is launched, Connected Content will send emails to the users that belong in the target group.  These emails will be formatted using the Personalize Campaign you created earlier.

Recommended Time: 1 hour

## Prerequisites

This module uses Amazon Personalize to generate and associate personalized product recommendations for users.  The content of this workshop is presented with the assumption that you have either completed the [Personalization](../1-Personalization/1.1-Personalize.ipynb) workshop or those resources have been pre-provisioned in your AWS environment. If you are unsure and attending an AWS managed event such as a workshop, check with your event lead.

This part of the workshop will also require access to a Braze account.  If you are unsure of how to set that up contact <TODO: INSERT CONTACT HERE>.

## Architecture

Before you set up Braze to send personalized messages to users, let's review the relevant parts of the Retail Demo Store architecture and how it uses Braze to integrate with the machine learning campaigns created in Personalize.

![Braze Personalization Architecture](images/braze-personalize-arch.png)

Braze will send emails to your users based on their behavior or based on attributes of their user profiles.  A discussion of Braze campaigns is beyond the scope of this document but Braze will ingest real-time events from mobile and web applications.  This data can be used to identify users and to build user profiles that can be used to determine when to message or email users.  

This event data flow will happen in parallel to the same behavioral event data being sent to Amazon Personalize. In this workshop, the demo store uses Amplify to send events to Personalize as shown in the diagram. This data is used to train a recommendations model that can in turn be used by Braze Connected Content to personalize content to users of your mobile and web applications when your Braze campaign runs.  

Braze Connected content will be able to get these recommendations via a recommendation service running in AWS.  Earlier in this workshop, this service was deployed using by the Cloud Formation templates you used to deploy the workshop environment in ECS.

### Building a Braze Email Template

Before you can run a campaign in Braze, you will need to create an email template that uses Braze Connected Content to fetch Product recommendations for the users you will be emailing via the campaign.

Log in to the Braze [console](https://dashboard.braze.com/auth).  You will need to have a Braze account configured for you to continue this workshop, or use your existing Braze account.

![Braze Login Screen](images/braze-create-template-0.png)

On the left hand pane of the Braze console, click the Templates & Media link.  This will show the Templates list screen.  Click the From File button under Basic Email Templates at the top of the screen.

![](images/braze-create-template-1.png)

A sample HTML template file has been provided for you in this workshop, to save editing time.  You can edit this template as needed but it is recommended that you follow this workshop at least once to faimilarize yourself with how this integration will work.  Click the Upload from File button:

![](images/braze-create-template-2.png)

Navigate to your local clone of this repository, to the `/workshop/4-Messaging/` folder.  Select the `braze-connected-content-email-template.zip` file and click the Open or Save button to upload the ZIP file to Braze

![](images/braze-create-template-3.png)

Braze will validate the file, and return a status window.  Click the Build Email button to continue

![](images/braze-create-template-4.png)

On the next screen, give the template a name, such as `Braze Connected Content Workshop Template`

![](images/braze-create-template-5.png)

On the bottom right of the screen, click the Save Template button.  Make sure to click the Save button often, the editor will not auto-save your changes.

![](images/braze-create-template-6.png)

Scroll back up and click the Edit Email Body link

![](images/braze-create-template-7.png)

This will take you to the HTML template editor.  At the very top, there is a Connected Content call that calls the Personalize recommendation service you deployed eariler, and gets a set of recommendations for the user ID that this template will be sent to:

```
{% connected_content <URL OF YOUR RECOMMENDATION SERVICE>/recommendations?userID={{${user_id}}}&fullyQualifyImageUrls=1&numResults=4 :save result %}
```

In order to complete the template edit, you will need to have the name of the load balancer for the Retail Demo Store recommendations service you deployed at the beginning of the Retail Demo Store workshop.  The following code will query the AWS account you are using for this workshop, get a list of load balancers, and find the one that belongs to the Retail Demo Store and has the tag `recommendations`.

In [2]:
import boto3

elbv2 = boto3.client('elbv2')
 
recommendations_elb_domain_name = None
 
elbs_paginator = elbv2.get_paginator('describe_load_balancers')
for elbs_page in elbs_paginator.paginate():
    for elb in elbs_page['LoadBalancers']:
        tags_response = elbv2.describe_tags(ResourceArns = [ elb['LoadBalancerArn'] ])
        for tag_desc in tags_response['TagDescriptions']:
            for tag in tag_desc['Tags']:
                if tag['Key'] == 'RetailDemoStoreServiceName' and tag['Value'] == 'recommendations':
                    recommendations_elb_domain_name = elb['DNSName']
                    break
            if recommendations_elb_domain_name:
                break
        if recommendations_elb_domain_name:
            break
    if recommendations_elb_domain_name:
        break
 
assert recommendations_elb_domain_name is not None, 'Unable to find Recommendations service ELB'
 
print('Recommendations DNS name: {}'.format(recommendations_elb_domain_name))

Recommendations DNS name: Retai-LoadB-S2PYVFN8F1GQ-1510179905.us-east-1.elb.amazonaws.com


Copy the Recommendations DNS name value you got from the code above.

Once you have that, you will need to enter the following code into the top of the email template, with <Recommendations DNS Name> replaced with the value you copied from above.

```
{% connected_content http://<Recommendations DNS Name>/recommendations?userID={{${user_id}}}&fullyQualifyImageUrls=1&numResults=4 :save result %}```

Do not change the URL query parameters.  This will break the template in the next few steps.

This snippet will be processed by Braze when you run your campaign, and Connected Content will query your Personalize recommendation service with the `user_id` this email is intended for, fetch recommendations, and then will fill in the appropriate sections of the HTML template with information about recommended products.

![](images/braze-create-template-8.png)

Note that this template is hard-coded to show 4 product recommendations.  Once Connected Content loads recommendations, they are available in the `result` array and they can be referenced in the HTML for your template.

When you are finished modifying your template, click the Done button at the bottom right of the editor screen.

![](images/braze-create-template-9.png)

Before navigating away from the template editor, make sure to click the Save Template button on the bottom right of the template window, or your earlier changes will be lost.

![](images/braze-create-template-10.png)

### What Does the Recommendations Service Return?

In order to understand how the Connected Content template works, it is helpful to take a look at the responses that are returned from the recommendations service.  

The code below will invoke your recommendations service, and return a response for one of the users that exists in the Reatail Demo Store user database.  You can change the `user_id` below to any valid ID.


In [7]:
import requests
import json

user_id = 15
response = requests.get('http://{}/recommendations?userID={}&fullyQualifyImageUrls=1&numResults=4'.format(recommendations_elb_domain_name, user_id))
#print(json.dumps(response.json(), indent = 2))
print(response.content)

b'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>404 Not Found</title>\n</head><body>\n<h1>Not Found</h1>\n<p>The requested URL /recommendations was not found on this server.</p>\n</body></html>\n'


### Upload Braze User Profiles

Before you can run a campaign from Braze, you will need to create a set of demo user profiles with valid emails in Braze. For this workshop, we recommend using a set of test emails that (preferably) map to your own email address or to an email address that you can control.

Assuming your **valid** email address is `joe@example.com`, add a few more variations using `+` notation such as:

- `joe+1@example.com`
- `joe+2@example.com`
- `joe+3@example.com`

To create the email profiles, you will need to edit the `braze-templates/test-users.csv` file in this directory.  Each row in this file corresponds to a user profile that will be created in Braze.

The file contents look like this:

```
external_id, first_name, last_name, email
33, Larry, Bird, joe+33@example.com
42, James, Worthy, joe+42@example.com
```

Modify the contents of the `email` column to correspond to the test email you will be using.  Note that the number after the `+` in each email must be different for each row.  After you make your changes, save your file and go back to the Braze console window and click the Upload Users link in the left hand side of the console.  Do not change the `external_id` column in the file.  This user ID needs to match a known user in the Retail Demo Store environment, because this identifier will be passed to the recommendation service in order to fetch recommendations for that user's email.

Note that if you are using a production Braze account for this workshop, you must be careful not to update existing user profiles with this upload, so it is a good idea to limit the number of rows in this example, and to make sure that you will not be using an `external_id` that corresponds to a real user profile.

When you are done with your edits, go to the Braze console, and click the User Import link on the left hand panel.  Then click the Select CSV File butotn. 

![](images/braze-upload-users-1.png)

This will open a file dialog.  Select the `test-users.csv` file you modified earlier, and click the Open button.

![](images/braze-upload-users-2.png)

Braze will analyze your CSV file. After a short time you will see a dialog that looks like this one:

![](images/braze-upload-users-3.png)

Click the Start Import button to begin the process of uploading users.  Braze will upload users in the background.  The status of the upload will be visible on this screen, and should complete quickly.

## Create a Target User Segment

Before you can run a campaign, you need to create a user segment.  A segment determines which users will receive emails from a campaign.

In the Braze console, click on Segments in the left pane, and then Create Segment in the top right corner of the window.

![](images/braze-create-segment-1.png)

Give your segment a name, like `Personalize Workshop`, and click the Create Segment button.

![](images/braze-create-segment-2.png)

Users can be matched to a segment by a variety of user attributes or behaviors.  For this workshop, you are going to use a regex that matches the emails of the users you uploaded in the previous section.

In the Filters section dropdow, scroll all the way to the bottom and select the email filter.  Then select matches regex for the matching criteria, and enter the regex `\+\d+@` into the text window. Braze should update the bottom of the window with a segment user count that matches the number of users in your upload file.  Click the Save button. 

![](images/braze-create-segment-3.png)

### Create and Run a Braze Campaign

In order to send personalized emails, you will need to configure a campaign.  In the Braze console, click the Campaigns link in the left hand menu pane. 

![](images/braze-campaign-1.png)

Select Create Campaign in the upper right corner of the screen; in the dropdown that appears, select Email for the campaign type.  For a more complete discussion of different Braze campaign types, check out their [campaign docs](https://www.braze.com/docs/user_guide/engagement_tools/campaigns/).

![](images/braze-campaign-2.png)

Enter the name of the campaign - something like `Braze Personalize Campaign`.

![](images/braze-campaign-3.png)

After entering the campaign name, scroll down to the bottom of the screen and select the Connected Content email template you created earlier in the workshop. 

![](images/braze-campaign-4.png)

After you select your template, scroll up and click the Edit Sending Info link.  Enter a subject line for your email.  Braze will not let you initiate a campaign without a subject line.

![](images/braze-campaign-5.png)

Click the Done button when you are done entering the subject line.

![](images/braze-campaign-6.png)

For this campaign, you will want emails to send immediately after the campaign launches, so select 'Send as soon as campaign is launched' under Time Based Scheduling Options.

![](images/braze-campaign-7.png)

Scroll down to the bottom of this pane.  This is optional but it may help in debugging any issues with getting emails from this campaign.  Under Delivery Controls, you will want to specify a short time period for when users in this campaign can be contacted again via email.  Braze will use this setting to prevent real users from being spammed by campaigns; in this workshop you are using your own email, so this should not prevent you from re-sending the campaign again if necessary.

Click the Forward link at the bottom of the screen.

![](images/braze-campaign-8.png)

Under Target Users, select the target user segment you created in the last step, then click the Forward link at the bottom of the screen.

![](images/braze-campaign-9.png)

You will not be using conversion events for this campaign, so you can click the Forward link on this screen.

![](images/braze-campaign-10.png)

At this point, you are ready to kick off the campaign.  Double check that the number of users in the campaign matches the number of users you added in your users CSV earlier in the workshop.  Then click the Launch Campaign button.  This will immediately start sending emails to the account that you specified in your CSV.

![](images/braze-campaign-11.png)

You should see an email in your inbox that looks something like this:

![](images/braze-email-example.png)

## Workshop Complete

Congratulations! You have completed the Retail Demo Store Braze Workshop.

### Cleanup

If you launched the Retail Demo Store in your personal AWS account **AND** you're done with all workshops & your evaluation of the Retail Demo Store, you can remove all provisioned AWS resources and data by deleting the CloudFormation stack you used to deploy the Retail Demo Store.

> If you are participating in an AWS managed event such as a workshop and using an AWS provided temporary account, you can skip the following cleanup steps unless otherwise instructed.